In [455]:
import copy
import time
import pickle
from collections import Counter 
from itertools import chain

import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable as Variable

import torchvision
import torchvision.transforms as transforms

In [456]:
def freq_dict(list):
    
    freq = {}
    for item in list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return(freq)

def clip(x, a=0, b=1):
    return(torch.max(torch.min(x, 0*x + b), 0*x + a))

In [488]:
def circuit_output(b, c, print_stuff=False):
#     print(b, c)
    
    n_strings = len(b[0])
    
    gate_i = 0
    cur_width = b.size()[0]
    
    while cur_width > 1:
        if print_stuff:
            print(cur_width)
            print(b)
        
        cur_width = round(cur_width/2)
        outputs = torch.zeros(cur_width, n_strings)

        for out_i in range(cur_width):
            in_i1 = out_i*2
            in_i2 = out_i*2 + 1

            outputs[out_i] = (1 - c[gate_i])*(1 - (1 - b[in_i1])*(1 - b[in_i2])) + c[gate_i]*b[in_i1]*b[in_i2]
            
            gate_i += 1

        b = outputs
    
    return(b[0])

In [500]:
def choose_target(n_bits):
    global c
    c = 1.*(torch.rand(n_bits - 1) > .5)

    while True:
        c = 1.*(torch.randn(n_bits - 1) > 0)
        outputs = [circuit_output(1.*(torch.randn(n_bits, 1) > 0), c) for i in range(1000)]
#         print(sum(outputs))
        if sum(outputs) > 400 and sum(outputs) < 600:
            break
            
    outputs = [circuit_output(1.*(torch.randn(n_bits, 1) > 0), c) for i in range(1000)]
    print("average output of", sum(outputs).item()/1000)
    print(c)
    return(c)

In [484]:
def generate_all_binary_vectors(n_bits, array_so_far=None):
    if array_so_far == None:
        array_so_far = []
#     print(n_bits, array_so_far)
    if n_bits == 0:
        return([array_so_far.copy()])
    
    outputs = []
    
    array_so_far += [0]
    outputs += generate_all_binary_vectors(n_bits - 1, array_so_far)
    
    array_so_far[len(array_so_far) - 1] = 1
    outputs += generate_all_binary_vectors(n_bits - 1, array_so_far)
    
    del array_so_far[-1]
    
    return(outputs)

## Initialize and optimize on hypercube - GD

In [498]:
def run_experiment(c, redundancy, n_epochs):
    print('#'*100)
    
    n_bits = len(c) + 1

    all_data = torch.tensor(generate_all_binary_vectors(n_bits))
    all_data_redundant = torch.cat((all_data,)*redundancy, dim=1)

    x = Variable(torch.rand(redundancy*n_bits - 1), requires_grad=True)
    optimizer = optim.SGD([x], lr=.003, momentum=.999)
    # optimizer = optim.Adam([x], lr=.1)

    for epoch in range(n_epochs):
    #     data = 1.*(torch.rand(10000, n_bits) > .5)
    #     data = torch.rand(10000, n_bits)
        data = all_data
        data_redundant = all_data_redundant

        optimizer.zero_grad()
        output_x = circuit_output(torch.t(data_redundant), x)
        output_c = circuit_output(torch.t(data), c)
        loss = torch.mean(torch.abs(output_x - output_c))

        loss.backward()
        optimizer.step()

        with torch.no_grad():
            x.data = clip(x.data)

        if epoch % 100 == 0:
            print('*'*100)
            print("loss: ", loss.item())

            x_snapped = 1.*(x > .5)
            output_s = circuit_output(torch.t(data_redundant), x_snapped)
            loss_snapped = torch.mean(torch.abs(output_s - output_c))
            print("loss snapped: ", loss_snapped.item())

            extremeness = torch.sum(torch.min(torch.abs(x), torch.abs(x - 1))).item()
            print("distance to nearest vertex: ", extremeness)
    
        if loss.item() == 0:
            break
        
    output_c = circuit_output(torch.t(data), c)
    x_snapped = 1.*(x > .5)
    output_s = circuit_output(torch.t(data_redundant), x_snapped)
    loss_snapped = torch.mean(torch.abs(output_s - output_c))
    
    print("final model snapped:", x_snapped)
    
    return loss_snapped.item()

In [534]:
lengths = [4, 8, 16]
cs = {}
for n_bits in lengths:
    cs[n_bits] = choose_target(n_bits)

average output of 0.557
tensor([0., 0., 1.])
average output of 0.538
tensor([1., 1., 1., 0., 0., 1., 0.])
average output of 0.497
tensor([1., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1.])


In [537]:
final_losses = {}
n_runs = 20
rdcies = [1, 2, 4, 8, 16, 32, 64]
for n_bits in lengths:
    c = cs[n_bits]
    for rdcy in rdcies:
        print()
        print()
        print('!'*100)
        print(n_bits, "bits, redundancy of", rdcy)
        final_losses[(n_bits, rdcy)] = [run_experiment(c, rdcy, 1000) for _ in range(n_runs)]



!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
4 bits, redundancy of 1
####################################################################################################
****************************************************************************************************
loss:  0.324795663356781
loss snapped:  0.25
distance to nearest vertex:  0.7016370296478271
****************************************************************************************************
loss:  0.2500273883342743
loss snapped:  0.25
distance to nearest vertex:  6.300475797615945e-05
****************************************************************************************************
loss:  0.2500115931034088
loss snapped:  0.25
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.25
loss snapped:  0.25
distance to nearest vertex:  0.0002475556975696236
****************

****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************

****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
final model snapped: tensor([0., 0., 1.])
####################################################################################################
****************************************************************************************************
loss:  0.3704379200935364
loss snapped:  0.375
distance to nearest vertex:  0.7958358526229858
****************************************************************************************************
loss:  0.2661880850791931
loss snapped:  0.375
distance to nearest vertex:  0.5670081973075867
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distan

****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
final model snap

****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
final model snapped: tensor([0., 0., 1.])
####################################################################################################
****************************************************************************************************
loss:  0.2615680992603302
loss snapped:  0.0
distance to nearest vertex:  1.0261540412902832
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
****************************************************************************************************
loss:  0.0
loss snapped:  0.0
distance to nearest vertex:  0.0
*******

...

****************************************************************************************************
loss:  0.44577720761299133
loss snapped:  0.446502685546875
distance to nearest vertex:  33.65959167480469
****************************************************************************************************
loss:  0.3966042697429657
loss snapped:  0.401611328125
distance to nearest vertex:  32.79159927368164
****************************************************************************************************
loss:  0.2192767858505249
loss snapped:  0.289642333984375
distance to nearest vertex:  26.53017234802246
****************************************************************************************************
loss:  0.045707084238529205
loss snapped:  0.043670654296875
distance to nearest vertex:  17.16712188720703
****************************************************************************************************
loss:  0.03758209943771362
loss snapped:  0.03790283203125
distance to 

****************************************************************************************************
loss:  0.46300190687179565
loss snapped:  0.516876220703125
distance to nearest vertex:  31.248023986816406
****************************************************************************************************
loss:  0.41821932792663574
loss snapped:  0.468841552734375
distance to nearest vertex:  31.014362335205078
****************************************************************************************************
loss:  0.2755790054798126
loss snapped:  0.140625
distance to nearest vertex:  27.123533248901367
****************************************************************************************************
loss:  0.02609417587518692
loss snapped:  0.0208740234375
distance to nearest vertex:  20.448532104492188
****************************************************************************************************
loss:  0.02104775235056877
loss snapped:  0.0208740234375
distance to neare

final model snapped: tensor([0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 1., 1., 1., 0., 0., 1.,
        1., 0., 1., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        1., 0., 1., 0., 1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        0., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 1., 1., 1.,
        1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        1., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1., 0., 0.,
        0., 1., 1.,

loss snapped:  0.471954345703125
distance to nearest vertex:  64.3436279296875
****************************************************************************************************
loss:  0.4088318943977356
loss snapped:  0.4683837890625
distance to nearest vertex:  61.861080169677734


KeyboardInterrupt: 

In [574]:
print(torch.Tensor(final_losses[(4, 1)]))
print(torch.Tensor(final_losses[(8, 1)]))
print(torch.Tensor(final_losses[(16, 1)]))

tensor([0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.2500, 0.0000, 0.0000,
        0.0000, 0.0000])
tensor([0.0000, 0.1328, 0.0000, 0.2812, 0.0000, 0.2031, 0.1328, 0.0000, 0.2031,
        0.2031, 0.1328, 0.0000, 0.2031, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.1328])
tensor([0.0000, 0.0209, 0.2911, 0.0209, 0.1378, 0.1281, 0.1960, 0.0000, 0.1960,
        0.2089, 0.1281, 0.1438, 0.0000, 0.2911, 0.0209, 0.0000, 0.0000, 0.1438,
        0.2911, 0.2089])


In [571]:
for params in sorted(final_losses):
    print(params[0], "bits and redundancy of", params[1], "gave average loss of", "%.5f"%torch.Tensor(final_losses[params]).mean().item(), "giving nonzero loss with probability", "%.2f"%(1 - sum(torch.Tensor(final_losses[params]) == 0).item()*1.0/n_runs))
    
for params in sorted(final_losses):
    print(params[0], params[1], "%.5f"%torch.Tensor(final_losses[params]).mean().item(), "%.2f"%(1 - sum(torch.Tensor(final_losses[params]) == 0).item()*1.0/n_runs))

4 bits and redundancy of 1 gave average loss of 0.03750 giving nonzero loss with probability 0.15
4 bits and redundancy of 2 gave average loss of 0.01250 giving nonzero loss with probability 0.05
4 bits and redundancy of 4 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 8 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 16 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 32 gave average loss of 0.00000 giving nonzero loss with probability 0.00
4 bits and redundancy of 64 gave average loss of 0.00000 giving nonzero loss with probability 0.00
8 bits and redundancy of 1 gave average loss of 0.08125 giving nonzero loss with probability 0.45
8 bits and redundancy of 2 gave average loss of 0.00742 giving nonzero loss with probability 0.10
8 bits and redundancy of 4 gave average loss of 0.00352 giving nonzero loss with probability 0.15
8 bits and redund

In [106]:
print(x)
y = torch.Tensor([1, 0, 0, 0, 0, 0, 0, 0])

output_x = circuit_output(torch.t(data_doubled), x)
output_y = circuit_output(torch.t(data_doubled), y)
output_c = circuit_output(torch.t(data), c)

print(sum(abs(output_x - output_c)))
print(sum(abs(output_y - output_c)))

# print(data_doubled)
# print(output_y)
# print(data_doubled[4:5])
# print('*'*100)
# print(circuit_output(torch.t(data_doubled[4:5]), y, print_stuff=True))

tensor([ 1.0000e+00, -5.4456e-06,  4.9505e-04,  7.5124e-01,  6.1709e-01,
         9.3323e-01,  8.1097e-01], requires_grad=True)
tensor(5.7244, grad_fn=<AddBackward0>)
tensor(6.)


In [135]:
x = torch.Tensor([0, 1, 0])

output_x = circuit_output(torch.t(data), x)
output_c = circuit_output(torch.t(data), c)

print(sum(abs(output_x - output_c)))

tensor(4.)


In [778]:
print(x)
print(c)

tensor([6.8733e-04, 1.0004e+00, 1.3043e-02], requires_grad=True)
tensor([0., 0., 1.])


In [385]:
x_orig = x
x_clipped = 1.*(x > .5)
x

tensor([0.9105, 0.6759, 1.0000, 0.9095, 0.0000, 0.0000, 1.0000, 1.0000, 0.0236,
        0.0000, 0.0000, 0.0000, 0.9949, 0.9949, 0.9748, 0.9748, 0.3941, 1.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.9849, 0.0000, 1.0000, 1.0000, 0.0000,
        0.0000, 0.0000, 0.8900, 1.0000], requires_grad=True)

In [699]:
data = all_data

x_tweaked = x_clipped + torch.tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
print(x_tweaked)

output_x = circuit_output(torch.t(data), x_clipped)
output_x_t = circuit_output(torch.t(data), x_tweaked)
output_c = circuit_output(torch.t(data), c)

loss = torch.mean(torch.abs((output_x - output_c)))
loss_t = torch.mean(torch.abs((output_x_t - output_c)))

print(loss_t - loss)

tensor([0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 1.])
tensor(0.)


In [667]:
c - x_clipped

tensor([ 0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.])

In [ ]:
# -> there are local mins besides the global min

In [852]:
# check if adding more gates besides AND/OR can help escape a local minimum

all_data = generate_all_binary_vectors(4)

def target_f(x):
    o1 = x[0] or x[1]
    o2 = x[2] or x[3]
    o3 = o1 and o2
    return(o3)

def current_f(x):
    o1 = x[0] or x[1]
    o2 = .8*(x[2] and x[3]) + .2*(x[2] or x[3])
    o3 = .99*o1 + .01*(o1 and o2)
    return(o3)

sum([abs(target_f(x) - current_f(x)) for x in all_data])

3.018

## Check for local minima

In [339]:
def bitstring_neighbors(b):
    neighbors = torch.cat((b,)*len(b[0]))
    neighbors = neighbors + torch.eye(len(b[0]))
    neighbors %= 2
    return(neighbors)

def binary_to_decimal(b):
    nb = len(b[0])
    x = torch.Tensor([[2**k for k in range(nb - 1, -1, -1)]])
    output = torch.mm(1.*b, 1.*torch.t(x))
    return(output.long())

def is_local_min(b, errors, strict=True):
    error = errors[binary_to_decimal(b).view(-1)[0]]
    neighbors = bitstring_neighbors(b)
    neighbor_is = binary_to_decimal(neighbors).view(-1).tolist()
    neighbor_errors = errors[neighbor_is]
    min_neighbor_error = min(neighbor_errors)
    
    if error < min_neighbor_error:
        return(True)
    elif error == min_neighbor_error:
        if strict:
            return(False)
        return(True)
    return(False)

def get_equivalent_neighbors_if_min(b, errors):
    error = errors[binary_to_decimal(b).view(-1)[0]]
    neighbors = bitstring_neighbors(b)
    neighbor_is = binary_to_decimal(neighbors).view(-1).tolist() + binary_to_decimal(b)[0].tolist()
    neighbor_errors = errors[neighbor_is]
    min_neighbor_error = min(neighbor_errors)
    
    if min_neighbor_error < error:
        return(set())
    
    output = set()
    for i in range(len(neighbor_is)):
        if neighbor_errors[i] == error:
            output.add(neighbor_is[i])
    return(output)

In [579]:
n_bits = 16
redundancy = 1
n_gates = redundancy*n_bits - 1

c = cs[n_bits]
# c = torch.Tensor([0, 0, 1])
# c = torch.Tensor([0, 0, 0, 0, 1, 1, 0])
# c = torch.Tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1])

data = torch.tensor(generate_all_binary_vectors(n_bits))
if redundancy > 1:
    data = torch.cat((data,)*redundancy, dim=1)

circuits = torch.tensor(generate_all_binary_vectors(n_gates))

errors = torch.zeros(len(circuits))
for i in range(len(circuits)):
    output = circuit_output(torch.t(data), circuits[i])
    target = circuit_output(torch.t(data), c)
    errors[i] = sum(abs(output - target))

print(errors)

KeyboardInterrupt: 

### Find single local mins

In [ ]:
for i in range(len(circuits)):
    if is_local_min(circuits[i:(i + 1)], errors, strict=True):
        print(i, circuits[i], errors[i])

### Find local min clusters

In [ ]:
cohort_is = [get_equivalent_neighbors_if_min(circuits[i:(i + 1)], errors) for i in range(len(circuits))]

original_cohort_is = copy.deepcopy(cohort_is)

for run in range(20):
    for i in range(len(circuits)):
        set_i = cohort_is[i]
        for j in set_i:
            if cohort_is[j] == set():
                cohort_is[i] = set()
                break

cluster_ids = [str(i) for i in range(len(circuits))]
for run in range(20):
    for i in range(len(circuits)):
        if cohort_is[i] == set():
            cluster_ids[i] = ''
        else:
            for j in cohort_is[i]:
                cluster_ids[j] = cluster_ids[i]

zero_cluster_ids = set()
min_cluster_ids = set()
for ci in set(cluster_ids):
    if ci != '':
        if errors[int(ci)] == 0:
            zero_cluster_ids.add(ci)
        else:
            min_cluster_ids.add(ci)

cluster_size_dict = freq_dict(cluster_ids)
min_cluster_errors = [errors[int(ci)].item() for ci in min_cluster_ids]
min_cluster_sizes = [cluster_size_dict[mci] for mci in min_cluster_ids]
zero_cluster_sizes = [cluster_size_dict[zci] for zci in zero_cluster_ids]
            
print(str(len(set(zero_cluster_ids))) + " zero clusters with sizes " + str(zero_cluster_sizes))
print(str(len(set(min_cluster_ids))) + " min clusters with sizes " + str(min_cluster_sizes) + " and errors " + str(min_cluster_errors))

print(cluster_ids)
print(cohort_is)


In [358]:
original_cohort_is

[{0, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384},
 {0, 1, 3, 5, 9, 17, 33, 65, 129, 257, 513, 1025, 2049, 4097, 8193, 16385},
 {0, 2, 3, 6, 10, 18, 34, 66, 130, 258, 514, 1026, 2050, 4098, 8194, 16386},
 set(),
 {0, 4, 5, 6, 12, 20, 36, 68, 132, 260, 516, 1028, 2052, 4100, 8196, 16388},
 set(),
 {2, 4, 6, 7, 14, 22, 38, 70, 134, 262, 518, 1030, 2054, 4102, 8198, 16390},
 set(),
 {0, 8, 9, 10, 12, 24, 40, 72, 136, 264, 520, 1032, 2056, 4104, 8200, 16392},
 set(),
 set(),
 {11, 15, 27, 43, 75, 523, 1035, 2059, 4107, 8203, 16395},
 {4, 8, 12, 13, 14, 28, 44, 76, 140, 268, 524, 1036, 2060, 4108, 8204, 16396},
 set(),
 set(),
 {11, 15, 31, 47, 79, 527, 1039, 2063, 4111, 8207, 16399},
 {0, 16, 17, 18, 20, 24, 48, 80, 144, 272, 528, 1040, 2064, 4112, 8208, 16400},
 set(),
 set(),
 {19, 23, 27, 51, 83, 147, 275, 2067, 4115, 8211, 16403},
 {4, 16, 20, 21, 22, 28, 52, 84, 148, 276, 532, 1044, 2068, 4116, 8212, 16404},
 set(),
 set(),
 {19, 23, 31, 55, 87, 151, 279, 2071